In [17]:
import os
import getpass

import pandas as pd
import matplotlib.pyplot as plt

import requests
import json


import asyncio

from typing import Sequence

import autogen
from autogen_core.models import UserMessage
from autogen_core.tools import FunctionTool
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination, TimeoutTermination, MaxMessageTermination
from autogen_agentchat.messages import AgentEvent, ChatMessage, TextMessage, MultiModalMessage
from autogen_agentchat.teams import RoundRobinGroupChat, SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_agentchat.base import TaskResult
from autogen_core import CancellationToken

In [ ]:
# Load your OPENAI API key
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

# Load your Spoonacular API key
if not os.environ.get('SPOONACULAR_API_KEY'):
    os.environ['SPOONACULAR_API_KEY'] = getpass.getpass("Enter your Spoonacular API key: ")

In [ ]:
# Create OpenAI model client for inference
openai_model_client = OpenAIChatCompletionClient(
    model = "gpt-4o-mini",
    api_key = os.environ.get("OPENAI_API_KEY")
)

SPOONACULAR_API_BASE_URL = f'https://api.spoonacular.com/recipes/complexSearch'

In [13]:
# Define all model tools
async def search_web_tool(query: str) -> str:
    """Searches web for relevant information."""
    # Call API!
    return "The start was 10 and the end was 15."

def percentage_change_tool(start: float, end: float) -> float:
    """Finds the percentage change between start and end."""
    return ((end - start) / start) * 100

def addition_tool(first: float, second: float) -> float:
    """Adds two numbers together."""
    return first + second

In [14]:
user_agent = UserProxyAgent(
    name = "UserProxyAgent",
    description = "Proxy agent for user interactions.",
)

web_search_agent = AssistantAgent(
    name = "WebSearchAgent",
    model_client = openai_model_client,
    description = "A web search agent for relevant results.",
    tools = [search_web_tool],
    system_message = """
        You are a web search agent.
        Your only tool is search_web_tool - use it to find information.
        You make only one search call at a time.
        Once you have the results, you never do calculations based on them.
    """
)

data_manipulator_agent = AssistantAgent(
    name = "DataManipulatorAgent",
    model_client = openai_model_client,
    description = "A data manipulator agent. Useful for performing calculations and ranking items according to relevance.",
    tools = [percentage_change_tool, addition_tool],
    system_message = """
        You are an expert data manipulator agent.
        Given the tasks you have been assigned, you should analyze the data and provide results.
    """
)

writer_agent = AssistantAgent(
    name = "WriterAgent",
    model_client = openai_model_client,
    description = "A writer agent. All writing should be in English, make sense, and be perfectly clear.",
    system_message = """
        You are an expert writer, on par with the best in the world.
        Given the data that you have received, you should write a full, cohesive, and ranked report about it that is directly in line with the user's request.
        You should emphasize and display adherence to all user filters that have been requested.
    """
)

In [15]:
# Set up the multi-agent team, rotation, and termination conditions

text_mention_termination = TextMentionTermination("APPROVE")
max_messages_termination = MaxMessageTermination(max_messages = 25)
# timeout_termination = TimeoutTermination(timeout_seconds = 10)

tm_termination = text_mention_termination | max_messages_termination

team = RoundRobinGroupChat(
    participants = [user_agent, web_search_agent, data_manipulator_agent, writer_agent], 
    termination_condition = text_mention_termination
)

In [16]:
# Run a task through the multi-agent system
await team.reset()

task = ""

await Console(team.run_stream(task = task))

---------- user ----------

---------- UserProxyAgent ----------
APPROVE


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='', type='TextMessage'), UserInputRequestedEvent(source='UserProxyAgent', models_usage=None, metadata={}, request_id='7932e8a7-8531-4d8b-906b-cc01a3e1b86e', content='', type='UserInputRequestedEvent'), TextMessage(source='UserProxyAgent', models_usage=None, metadata={}, content='APPROVE', type='TextMessage')], stop_reason="Text 'APPROVE' mentioned")